In [1]:
using MKL
using Revise
using LinearAlgebra
using SparseArrays
using Plots
using ReducedBasis

Construct the $S=1/2$ XXZ Hamiltonian
$$
H(\mathbf{\mu}) = \sum_{i=1}^{L-1} [S_i^x S_{i+1}^x + S_i^y S_{i+1}^y
+ \Delta S_i^z S_{i+1}^z ] - \frac{h}{J} \sum_{i=1}^L S_i^z
$$
with the parameter vector $\mathbf{\mu} = (\Delta,\, h/J)$. The many-body spin operators $S_i^\gamma = (\otimes^{i-1} I) \otimes \frac{1}{2}\sigma^\gamma \otimes (\otimes^{L-i} I)$ are constructed from the $2 \times 2$ Pauli matrices
$$
\sigma^x = \begin{bmatrix} 0 & 1 \\ 1 & 0 \end{bmatrix}, \quad
\sigma^y = \begin{bmatrix} 0 & -i \\ i & 0 \end{bmatrix}, \quad
\sigma^z = \begin{bmatrix} 1 & 0 \\ 0 & -1 \end{bmatrix}.
$$

In [2]:
# Define (dense) Pauli matrices
σx = [0.0 1.0; 1.0 0.0]
σy = [0.0 -im; im 0.0]
σz = [1.0 0.0; 0.0 1.0]

# Convert local-site to many-body operator
function local_to_global(L::Int, op::M, i::Int) where M <: AbstractMatrix
    d = size(op, 1)
    @assert d == size(op, 2) "Operator has to be a square matrix."

    if i == 1
        return kron(op, M(I, d^(L - 1), d^(L - 1)))
    elseif i == L
        return kron(M(I, d^(L - 1), d^(L - 1)), op)
    else
        return kron(
            kron(M(I, d^(i - 1), d^(i - 1)), op), M(I, d^(L - i), d^(L - i))
        )
    end
end

local_to_global (generic function with 1 method)

In the RBM framework we must formulate $H(\mathbf{\mu})$ in terms of an *affine decomposition*
$$
H(\mathbf{\mu}) = \sum_{q=1}^Q \theta_q(\mathbf{\mu})\, H_q
$$
where $H_q$ are Hermitian matrices of Hilbert space dimension and only the coefficients $\theta_q(\mathbf{\mu})$ are parameter-dependent.

In [3]:
function xxz_chain(L, ::Type{Matrix})
    H1 = 0.25 * sum([
        local_to_global(L, σx, i) * local_to_global(L, σx, i + 1) +
        local_to_global(L, σy, i) * local_to_global(L, σy, i + 1) for i = 1:L-1
    ])
    H2 = 0.25 * sum([
        local_to_global(L, σz, i) * local_to_global(L, σz, i + 1) for i = 1:L-1
    ])
    H3 = 0.5 * sum([local_to_global(L, σz, i) for i = 1:L])
    coefficient_map = μ -> [1.0, μ[1], -μ[2]]
    return AffineDecomposition([H1, H2, H3], coefficient_map)
end

xxz_chain (generic function with 1 method)

Next, we initialize all objects and parameters that are needed to assemble a reduced basis. This includes

- the model, initialized as an `AffineDecomposition` object
- the `Greedy` assembly strategy using RB errors estimation from the `Residual` struct corresponding to $\mathrm{Res}_n(\mathbf{\mu})$
- the truth solver for determining the *snapshots* at different parameter points, using `FullDiagonalization`
- the training grid of all parameter points to choose from, using `RegularGrid`

In [49]:
L = 8
H_XXZ = xxz_chain(L, Matrix)

greedy = Greedy(; estimator=Residual(), tol_residual=1e-3, n_truth_max=64)

solver = FullDiagonalization(;
    n_states_max=L+1, tol_degeneracy=1e-10, full_orthogonalize=false, tol_qr=1e-10
) # m = L + 1 degeneracy at (Δ, h/J) = (-1, 0)

Δ = range(-1.0, 2.5, 40)
h = range(0.0, 3.5, 40)
grid_train = RegularGrid(Δ, h)

40×40 RegularGrid{2}:
 [-1.0, 0.0]       [-1.0, 0.0897436]       …  [-1.0, 3.5]
 [-0.910256, 0.0]  [-0.910256, 0.0897436]     [-0.910256, 3.5]
 [-0.820513, 0.0]  [-0.820513, 0.0897436]     [-0.820513, 3.5]
 [-0.730769, 0.0]  [-0.730769, 0.0897436]     [-0.730769, 3.5]
 [-0.641026, 0.0]  [-0.641026, 0.0897436]     [-0.641026, 3.5]
 [-0.551282, 0.0]  [-0.551282, 0.0897436]  …  [-0.551282, 3.5]
 [-0.461538, 0.0]  [-0.461538, 0.0897436]     [-0.461538, 3.5]
 [-0.371795, 0.0]  [-0.371795, 0.0897436]     [-0.371795, 3.5]
 [-0.282051, 0.0]  [-0.282051, 0.0897436]     [-0.282051, 3.5]
 [-0.192308, 0.0]  [-0.192308, 0.0897436]     [-0.192308, 3.5]
 ⋮                                         ⋱  
 [1.78205, 0.0]    [1.78205, 0.0897436]       [1.78205, 3.5]
 [1.87179, 0.0]    [1.87179, 0.0897436]       [1.87179, 3.5]
 [1.96154, 0.0]    [1.96154, 0.0897436]       [1.96154, 3.5]
 [2.05128, 0.0]    [2.05128, 0.0897436]       [2.05128, 3.5]
 [2.14103, 0.0]    [2.14103, 0.0897436]    …  [2.14103, 3.5]
 

In [73]:
basis, h, info = assemble(
    H_XXZ, grid_train, greedy, solver; solver_online=solver
);

n      max. err    time      μ       
1      NaN         28.7ms    Number[-1.0, 0.0]


2      1.75e-06     556ms    Number[-0.5512820512820513, 3.41025641025641]
Reached residual target.


To finish the offline phase, we measure an observable. In this example we want to measure the magnetization
$$
M = \frac{2}{L} \sum_{i=1}^L S_i^z,
$$
for we which again construct an `AffineDecomposition` that only contains one term and one coefficent. Note that this term is already contained in $H(\mathbf{\mu})$.

In [ ]:
M = AffineDecomposition([H.terms[3]], μ -> [2 / L])
m = compress(M, basis)

The measurements in the online phase are performed by calling the reduced observables, which are just in `AffineDecomposition` form again. Due to the reduced computational effort, we can define a much finer grid now.

In [ ]:
Δ_online = range(first(Δ), last(Δ), 200)
h_online = range(first(h), last(h), 200)
grid_online = RegularGrid(Δ_online, h_online)

magnetization = Matrix{Float64}(undef, size(grid_online))
m_reduced = m([1])  # Save observable, since coefficients do not depend on μ 
for (idx, μ) in pairs(grid_online)
    λ_rb, φ_rb = online_solve(h, basis.metric, μ, solver)
    magnetization[idx] = sum(eachcol(φ_rb)) do φ
        dot(φ, m_reduced, φ) / size(φ_rb, 2)  # Divide by multiplicity
    end
end

In [ ]:
hm = heatmap(
    grid_online.ranges[1],
    grid_online.ranges[2],
    magnetization';  # transpose to have rows as x-axis
    title="\$\\langle M \\rangle\$",
    aspect_ratio=:equal,
    clims=(0.0, 1.0),
)
# plot!(hm, grid_online.ranges[1], x -> 1 + x; lw=2, ls=:dash, legend=false)
scatter!(
    hm,
    [μ[1] for μ in diagnostics.snapshot],
    [μ[2] for μ in diagnostics.snapshot];
    markershape=:xcross,
    color=:green,
    msw=2,
)